In [ ]:
"""
w/o bert toknizer 버전 
기존의 문장에서의 split 만을 사용하여 
BLEU-4 Score를 측정하였음, 

기존의 ARPER 논문의 BLEU-4 score를 정의하는게 
제대로 설명이 안되어있는데 
통상적으로 BLEU-4 Score는 4-GRAM짜리의 BLEU스코어를
의미하기도 하지만 
cumulative 1~4개 까지의 평균도 BLEU-4 라고 불림 

그래서 일단 두개의 코드를 다 적어둠
"""
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
# 환경변수 설정 구간 

#test.sh의 환경변수 그대로 
DATA_DIR= 'multiwoz2_data'
DOMAIN= '0_attraction'
dataset_path = "{}/{}".format(DATA_DIR, DOMAIN)
model_checkpoint = "./runs/latest/"

# Output file 파일명적기 
output_file = 'Oct08_03-53-49_output.txt'
output_file_path = model_checkpoint+output_file


In [ ]:
# test dataset load
with open("{}/{}_test.txt".format(DATA_DIR, DOMAIN), 'r') as f:
    golden_responses =  [] 
    lines = f.readlines()
    for line in lines:
        _, a = line.split('|', maxsplit=1)
        golden_responses.append(a.strip().split())

# predict dataset load
with open(output_file_path, 'r') as f:
    predict_responses = []
    lines = f.readlines()
    for line in lines:
        predict_responses.append(line.strip().split())    

assert len(golden_responses) == len(predict_responses)

In [ ]:
# For DEBUG
# 실행안해두됨
"""
https://github.com/alvations/nltk/blob/develop/nltk/translate/bleu_score.py#L425

class SmoothingFunction:
This is an implementation of the smoothing techniques
for segment-level BLEU scores that was presented in
Boxing Chen and Collin Cherry (2014) A Systematic Comparison of
Smoothing Techniques for Sentence-Level BLEU. In WMT14.
http://acl2014.org/acl2014/W14-33/pdf/W14-3346.pdf
"""
for golden, predict in zip(golden_responses, predict_responses):
    print("G : {}".format(golden))
    print("P : {}".format(predict))
    print('Individual 1-gram: %f' % sentence_bleu(golden, predict, weights=(1, 0, 0, 0)))
    print('Individual 2-gram: %f' % sentence_bleu(golden, predict, weights=(0, 1, 0, 0), smoothing_function=SmoothingFunction().method4))
    print('Individual 3-gram: %f' % sentence_bleu(golden, predict, weights=(0, 0, 1, 0), smoothing_function=SmoothingFunction().method4))
    print('Individual 4-gram: %f' % sentence_bleu(golden, predict, weights=(0, 0, 0, 1), smoothing_function=SmoothingFunction().method4))
    print('cumulative 4-gram: %f' % sentence_bleu(golden, predict, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=SmoothingFunction().method4))

In [ ]:
# BLEU Score 계산부분
BLEU_scores = [0, 0, 0, 0, 0]

for golden, predict in zip(golden_responses, predict_responses):
    blue1 = sentence_bleu(golden, predict, weights=(1, 0, 0, 0))
    blue2 = sentence_bleu(golden, predict, weights=(0, 1, 0, 0), smoothing_function=SmoothingFunction().method4)
    blue3 = sentence_bleu(golden, predict, weights=(0, 0, 1, 0), smoothing_function=SmoothingFunction().method4)
    blue4 = sentence_bleu(golden, predict, weights=(0, 0, 0, 1), smoothing_function=SmoothingFunction().method4)
    blue4_cum = sentence_bleu(golden, predict, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=SmoothingFunction().method4)
    
    BLEU_scores[0] += blue1
    BLEU_scores[1] += blue2
    BLEU_scores[2] += blue3
    BLEU_scores[3] += blue4
    BLEU_scores[4] += blue4_cum

print('Individual 1-gram: ', BLEU_scores[0] / len(predict_responses))
print('Individual 2-gram: ', BLEU_scores[1] / len(predict_responses))
print('Individual 3-gram: ', BLEU_scores[2] / len(predict_responses))
print('Individual 4-gram: ', BLEU_scores[3] / len(predict_responses))
print('cumulative 4-gram: ', BLEU_scores[4] / len(predict_responses))